In [9]:
# --- Versión ajustada: permitir más unidades, tolerancia más amplia, más generaciones ---

import math
import random
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms

# ---------------------------------------------
# 1) Definir tabla de productos con costos
# ---------------------------------------------
products_df = pd.DataFrame.from_records([
    ['Banano 1u', 0, 4, 89, 1, 0, 23, 0.3],
    ['Mandarina 1u', 0, 4, 40, 1, 0, 10, 0.25],
    ['Piña 100g', 0, 7, 50, 1, 0, 13, 0.4],
    ['Uvas 100g', 0, 7, 76, 1, 0, 17, 0.6],
    ['Chocolate 1 bar', 0, 4, 230, 3, 13, 25, 1.5],

    ['Queso Paipa 100g', 0, 8, 350, 28, 26, 2, 2.0],
    ['Quesillo 100g', 0, 8, 374, 18, 33, 1, 1.8],
    ['Pesto 100g', 0, 8, 303, 3, 30, 4, 1.2],
    ['Hummus 100g', 0, 8, 306, 7, 25, 11, 1.0],
    ['Pasta de berenjena 100g', 0, 4, 228, 1, 20, 8, 0.9],

    ['Batido de proteinas', 0, 5, 160, 30, 3, 5, 2.5],
    ['Hamburguesa vegetariana 1', 0, 5, 220, 21, 12, 3, 2.0],
    ['Hamburguesa vegetariana 2', 0, 12, 165, 16, 9, 2, 1.8],
    ['Huevo cocido 1', 0, 8, 155, 13, 11, 1, 0.2],
    ['Huevo frito 1', 0, 16, 196, 14, 15, 1, 0.25],

    ['Medio baguette', 0, 3, 274, 10, 0, 52, 1.0],
    ['Pan tajado 1 tajada', 0, 3, 97, 3, 1, 17, 0.3],
    ['Pizza de queso 1u', 0, 3, 903, 36, 47, 81, 4.0],
    ['Pizza vegetariana 1u', 0, 3, 766, 26, 35, 85, 3.5],

    ['Leche de soya 200ml', 0, 1, 115, 8, 4, 11, 0.8],
    ['Leche de soya achocolatada 250ml', 0, 3, 160, 7, 6, 20, 1.0],
])

products_df.columns = ['Nombre', 'Min', 'Max', 'Calorias', 'Gram_Prot', 'Gram_Grasa', 'Gram_Carb', 'Costo']

# ---------------------------------------------
# 2) Extraer datos numéricos para el algoritmo
# ---------------------------------------------
cal_data_list = list(products_df['Calorias'].astype(float))
prot_data = list(products_df['Gram_Prot'].astype(float))
fat_data = list(products_df['Gram_Grasa'].astype(float))
carb_data = list(products_df['Gram_Carb'].astype(float))
cost_data = list(products_df['Costo'].astype(float))

N_PRODUCTS = len(products_df)
print(f"Productos detectados: {N_PRODUCTS}. Con columna de costos incluida.")

# ---------------------------------------------
# 3) Definir metas nutricionales (ejemplo si no existen)
# ---------------------------------------------
try:
    target_prot_g = gram_prot
    target_fat_g  = gram_fat
    target_carb_g = gram_carb
except NameError:
    total_calories_week = 2500 * 7
    perc_prot = 0.30
    perc_carb = 0.50
    perc_fat  = 0.20
    target_prot_g = (total_calories_week * perc_prot) / 4.0
    target_carb_g = (total_calories_week * perc_carb) / 4.0
    target_fat_g  = (total_calories_week * perc_fat) / 9.0
    print("Metas nutricionales de ejemplo generadas.")

# ---------------------------------------------
# 4) Crear clases DEAP multiobjetivo
# ---------------------------------------------
try:
    creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))
except Exception:
    pass

try:
    creator.create("IndividualMulti", list, fitness=creator.FitnessMulti)
except Exception:
    pass

# Toolbox
toolbox = base.Toolbox()
MAX_UNITS = 30   # Aumentamos el límite de unidades por producto
toolbox.register("attr_int", random.randint, 0, MAX_UNITS)
toolbox.register("individual", tools.initRepeat, creator.IndividualMulti, toolbox.attr_int, n=N_PRODUCTS)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=MAX_UNITS, indpb=0.2)
toolbox.register("select", tools.selNSGA2)

# ---------------------------------------------
# 5) Función de evaluación
# ---------------------------------------------
NUTR_TOL = 0.20   # Aumentamos tolerancia a 20%
def evaluate_individual(individual):
    total_prot = sum(individual[i] * prot_data[i] for i in range(N_PRODUCTS))
    total_carb = sum(individual[i] * carb_data[i] for i in range(N_PRODUCTS))
    total_fat  = sum(individual[i] * fat_data[i] for i in range(N_PRODUCTS))
    total_cost = sum(individual[i] * cost_data[i] for i in range(N_PRODUCTS))

    eps = 1e-9
    err_prot = abs(total_prot - target_prot_g) / (target_prot_g + eps)
    err_carb = abs(total_carb - target_carb_g) / (target_carb_g + eps)
    err_fat  = abs(total_fat - target_fat_g) / (target_fat_g + eps)

    nutrition_error = err_prot + err_carb + err_fat
    return (nutrition_error, float(total_cost))

toolbox.register("evaluate", evaluate_individual)

# ---------------------------------------------
# 6) Algoritmo evolutivo
# ---------------------------------------------
def run_ea_multi(pop_size=200, ngen=200, cxpb=0.6, mutpb=0.3, seed=42):
    random.seed(seed)
    pop = toolbox.population(n=pop_size)
    invalid = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid)
    for ind, fit in zip(invalid, fitnesses):
        ind.fitness.values = fit

    for gen in range(1, ngen + 1):
        offspring = algorithms.varAnd(pop, toolbox, cxpb, mutpb)
        invalid = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid)
        for ind, fit in zip(invalid, fitnesses):
            ind.fitness.values = fit
        pop = toolbox.select(pop + offspring, pop_size)

        if gen % 20 == 0 or gen == 1:
            best = sorted(pop, key=lambda ind: (ind.fitness.values[0], ind.fitness.values[1]))[0]
            print(f"Gen {gen}: mejor individuo (nut_err, cost) = {best.fitness.values}")

    return pop

# ---------------------------------------------
# 7) Selección y presentación
# ---------------------------------------------
def select_cheapest_feasible(pop, nut_tol=NUTR_TOL):
    feasible = [ind for ind in pop if ind.fitness.values[0] <= nut_tol]
    if not feasible:
        print("No hay soluciones factibles dentro de la tolerancia.")
        return None
    cheapest = min(feasible, key=lambda ind: ind.fitness.values[1])
    return cheapest

def describe_diet(individual):
    q = np.array(individual)
    df = products_df.copy()
    df['Units'] = q
    df['Cost_total'] = df['Units'] * df['Costo']
    used = df[df['Units'] > 0].copy()
    display(used[['Nombre','Units','Costo','Cost_total','Gram_Prot','Gram_Carb','Gram_Grasa']])
    print("Costo total:", used['Cost_total'].sum())

# ---------------------------------------------
# 8) Ejemplo de ejecución
# ---------------------------------------------
if __name__ == '__main__':
    pop = run_ea_multi(pop_size=200, ngen=200)
    cheapest = select_cheapest_feasible(pop, nut_tol=NUTR_TOL)
    if cheapest is not None:
        describe_diet(cheapest)


Productos detectados: 21. Con columna de costos incluida.
Metas nutricionales de ejemplo generadas.
Gen 1: mejor individuo (nut_err, cost) = (6.048685714272869, 275.95)


/usr/local/lib/python3.12/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.12/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'IndividualMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Gen 20: mejor individuo (nut_err, cost) = (0.5830285714276532, 89.75)
Gen 40: mejor individuo (nut_err, cost) = (0.08546666666653216, 118.75)
Gen 60: mejor individuo (nut_err, cost) = (0.05066666666661527, 118.45)
Gen 80: mejor individuo (nut_err, cost) = (0.04811428571424473, 114.35)
Gen 100: mejor individuo (nut_err, cost) = (0.02925714285708971, 119.35)
Gen 120: mejor individuo (nut_err, cost) = (0.02925714285708971, 119.35)
Gen 140: mejor individuo (nut_err, cost) = (0.02925714285708971, 119.35)
Gen 160: mejor individuo (nut_err, cost) = (0.013180952380928564, 118.55)
Gen 180: mejor individuo (nut_err, cost) = (0.013180952380928564, 118.55)
Gen 200: mejor individuo (nut_err, cost) = (0.013180952380928564, 118.55)


,Nombre,Units,Costo,Cost_total,Gram_Prot,Gram_Carb,Gram_Grasa
0,Banano 1u,28,0.30,8.40,1,23,0
1,Mandarina 1u,1,0.25,0.25,1,10,0
2,Piña 100g,3,0.40,1.20,1,13,0
4,Chocolate 1 bar,1,1.50,1.50,3,25,13
9,Pasta de berenjena 100g,1,0.90,0.90,1,8,20
10,Batido de proteinas,20,2.50,50.00,30,5,3
13,Huevo cocido 1,9,0.20,1.80,13,1,11
14,Huevo frito 1,10,0.25,2.50,14,1,15
15,Medio baguette,22,1.00,22.00,10,52,0
16,Pan tajado 1 tajada,4,0.30,1.20,3,17,1


Costo total: 93.75
